Prvo pisanje kode za število podpoti v grafu, na primeru grafa n = 6

In [ ]:
# primer: generiraj vse povezane grafe z n=6
n = 6
vsi_grafi = [G for G in graphs(n) if G.is_connected()]
print("Število povezanih grafov:", len(vsi_grafi))

def cyclomatic_number(G):
    return G.size() - G.order() + G.connected_components_number()

k = 2
grafi_k = [G for G in vsi_grafi if cyclomatic_number(G) == k]
print("Število grafov z µ(G)=2:", len(grafi_k))

def all_simple_paths(G, start, end, visited=None):
    if visited is None:
        visited = []
    visited = visited + [start]
    if start == end:
        return [visited]
    paths = []
    for neighbor in G.neighbors(start):
        if neighbor not in visited:
            newpaths = all_simple_paths(G, neighbor, end, visited)
            for p in newpaths:
                paths.append(p)
    return paths

def subpath_number(G):
    n = 0
    V = list(G.vertices())
    for i in range(len(V)):
        for j in range(i, len(V)):
            n += len(all_simple_paths(G, V[i], V[j]))
    return n

for G in grafi_k:
    print(f"|V|={G.order()}, |E|={G.size()}, µ(G)={cyclomatic_number(G)}, p_n(G)={subpath_number(G)}")


Ta funkcija ti za grafe do n=8 in vse mu-je generira vse grafe in poišče tiste z njamanjši in največjim subpath numberjem, 
slike shrani v Slike_max_1del  in Slike_min_1del.
Rezulate pa zapiusuje je 1del_mali_shranjevanje_slik
Čas izvajanja kode je 43 s

In [1]:
import itertools
from sage.all import *
import pandas as pd
import os

# --------------------------------------------------
# Funkcija za ciklomatsko število
# --------------------------------------------------
def cyclomatic_number(G):
    return G.size() - G.order() + G.connected_components_number()

# --------------------------------------------------
# Funkcija za štetje enostavnih poti
# --------------------------------------------------
def all_simple_paths(G, start, end, visited=None):
    if visited is None:
        visited = []
    visited = visited + [start]
    if start == end:
        return [visited]
    paths = []
    for nei in G.neighbors(start):
        if nei not in visited:
            newp = all_simple_paths(G, nei, end, visited)
            paths.extend(newp)
    return paths

def subpath_number(G):
    V = G.vertices()
    total = 0
    for i in range(len(V)):
        for j in range(i, len(V)):
            total += len(all_simple_paths(G, V[i], V[j]))
    return total


# --------------------------------------------------
# Glavni izračun – z enoličnim zapisom grafov (graph6)
# --------------------------------------------------

os.makedirs("Slike_min_1del", exist_ok=True)
os.makedirs("Slike_max_1del", exist_ok=True)

rezultat_vrstice = []   # za CSV

for n in range(3, 9):
    print(f"\nObdelujem vse povezane grafe z n = {n} vozlišči ...")

    po_mu = {}   # mu : list of (pn, G)

    for G in graphs(n):
        if not G.is_connected():
            continue

        mu = cyclomatic_number(G)
        pn = subpath_number(G)

        if mu not in po_mu:
            po_mu[mu] = []
        po_mu[mu].append((pn, G))

    # min/max
    for mu in sorted(po_mu.keys()):
        seznam = sorted(po_mu[mu], key=lambda x: x[0])

        min_pn, G_min = seznam[0]
        max_pn, G_max = seznam[-1]

        # --- slike ---
        min_slika = f"Slike_min_1del/graf_min_n{n}_mu{mu}.png"
        max_slika = f"Slike_max_1del/graf_max_n{n}_mu{mu}.png"

        G_min.plot().save(min_slika)
        G_max.plot().save(max_slika)

        # --- enoličen zapis grafa (graph6) ---
        g6_min = G_min.graph6_string()
        g6_max = G_max.graph6_string()

        # --- zapišemo v tabelo ---
        rezultat_vrstice.append([n, mu, min_pn, max_pn,
                                 min_slika, max_slika,
                                 g6_min, g6_max])

print("\n--- Izračun končan ---")


# --------------------------------------------------
# CSV izpis
# --------------------------------------------------

df = pd.DataFrame(
    rezultat_vrstice,
    columns=[
        "n",
        "µ(G)",
        "min p_n(G)",
        "max p_n(G)",
        "slika_min",
        "slika_max",
        "graph6_min",
        "graph6_max"
    ]
)

df.to_csv("1del_mali_shranjevanje_slik.csv", index=False)



Obdelujem vse povezane grafe z n = 3 vozlišči ...

Obdelujem vse povezane grafe z n = 4 vozlišči ...

Obdelujem vse povezane grafe z n = 5 vozlišči ...

Obdelujem vse povezane grafe z n = 6 vozlišči ...

Obdelujem vse povezane grafe z n = 7 vozlišči ...

Obdelujem vse povezane grafe z n = 8 vozlišči ...

--- Izračun končan ---


Ta funkcija je podobna kot zgornja, le da ne shranjuje slik 
Kodo sva naredila do vklucno n=9 vozlisc
Prav tako sproti zapisuje podatke v csv
Podatki se shranjujejo v 1del_mali.csv

In [ ]:
from sage.all import *
import pandas as pd
import os

def cyclomatic_number(G):
    return G.size() - G.order() + G.connected_components_number()

def all_simple_paths(G, start, end, visited=None):
    if visited is None:
        visited = []
    visited = visited + [start]

    if start == end:
        return [visited]

    paths = []
    for nei in G.neighbors(start):
        if nei not in visited:
            for p in all_simple_paths(G, nei, end, visited):
                paths.append(p)
    return paths

def subpath_number(G):
    V = list(G.vertices())
    total = 0
    for i in range(len(V)):
        for j in range(i, len(V)):
            total += len(all_simple_paths(G, V[i], V[j]))
    return total


# ---------------- CSV INIT ----------------
CSV_FILE = "1del_mali.csv"

if not os.path.exists(CSV_FILE):
    pd.DataFrame(columns=["n","mu","min_pn","min_g6","max_pn","max_g6"])\
        .to_csv(CSV_FILE, index=False)


def save_live(n, mu, min_pn, min_g6, max_pn, max_g6):
    df = pd.read_csv(CSV_FILE)

    df = df[~((df["n"] == n) & (df["mu"] == mu))]

    df.loc[len(df)] = [
        int(n), int(mu),
        min_pn, min_g6,
        max_pn, max_g6
    ]

    df.to_csv(CSV_FILE, index=False)


# ---------------- GLAVNI PROGRAM ----------------
for n in range(3, 10):

    print(f"\n>>> ZAČENJAM n = {n}")

    # 1) ZBEREMO GRAFE PO µ
    bucket = {}
    for G in graphs(n):
        if not G.is_connected():
            continue
        mu = cyclomatic_number(G)
        bucket.setdefault(mu, []).append(G)

    # 2) OBRAVNAVAMO µ PO VRSTI — takojšnja obdelava + zapis
    for mu in sorted(bucket.keys()):
        print(f"\n*** OBRAVNAVAM µ = {mu} (n={n}) ***")

        best_min = None
        best_max = None

        for G in bucket[mu]:
            pn = subpath_number(G)
            g6 = G.graph6_string()

            # MIN
            if best_min is None or pn < best_min[0]:
                best_min = (pn, g6, G.copy())

            # MAX
            if best_max is None or pn > best_max[0]:
                best_max = (pn, g6, G.copy())

        # PO ZAKLJUČKU µ — takojšnji zapis!
        min_pn, min_g6, Gmin = best_min
        max_pn, max_g6, Gmax = best_max

        print(f"  MIN={min_pn}, MAX={max_pn}")
        Gmin.show()
        Gmax.show()

        save_live(n, mu, min_pn, min_g6, max_pn, max_g6)

